In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L4\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['Capital.csv', 'Central.csv', 'Dunwoody.csv', 'Genese.csv', 'Hudson.csv', 'LongIsland.csv', 'Milwood.csv', 'NYC.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[6]))

,Demand,Normalized_net,Temprature
0,336.5,0.525130,-5.8
1,317.7,0.495791,-6.1
2,307.0,0.479093,-7.5
3,300.6,0.469106,-8.4
4,298.4,0.465672,-7.4
...,...,...,...
43819,385.7,0.601910,3.4
43820,366.2,0.571479,3.3
43821,347.3,0.541984,2.6
43822,326.5,0.509524,2.7


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[6]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temprature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_Milwood,Temp_Milwood
0,336.5,-5.8
1,317.7,-6.1
2,307.0,-7.5
3,300.6,-8.4
4,298.4,-7.4
...,...,...
43819,385.7,3.4
43820,366.2,3.3
43821,347.3,2.6
43822,326.5,2.7


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_Milwood,Temp_Milwood,Lag-2,Lag-7
0,388.7,-13.9,333.6,336.5
1,379.0,-14.2,323.6,317.7
2,377.4,-14.8,317.7,307.0
3,377.9,-15.6,314.1,300.6
4,384.7,-16.0,320.7,298.4
...,...,...,...,...
43651,385.7,3.4,380.2,369.0
43652,366.2,3.3,375.4,364.1
43653,347.3,2.6,356.6,352.2
43654,326.5,2.7,328.1,341.5


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 14s 9ms/step - loss: 0.0122 - mape: 10879.9531 - mae: 0.0853 - val_loss: 0.0063 - val_mape: 23.3892 - val_mae: 0.0647
Epoch 2/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0073 - mape: 10893.2158 - mae: 0.0664 - val_loss: 0.0071 - val_mape: 21.1444 - val_mae: 0.0631
Epoch 3/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0050 - mape: 10811.0322 - mae: 0.0549 - val_loss: 0.0079 - val_mape: 22.0264 - val_mae: 0.0652
Epoch 4/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0044 - mape: 10527.7539 - mae: 0.0511 - val_loss: 0.0078 - val_mape: 22.7520 - val_mae: 0.0659
Epoch 5/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0040 - mape: 10571.2910 - mae: 0.0486 - val_loss: 0.0078 - val_mape: 18.4412 - val_mae: 0.0605
Epoch 6/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0037 - mape: 10454.6787 - mae: 0.0465 - val_loss: 

Epoch 48/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0020 - mape: 10212.0869 - mae: 0.0345 - val_loss: 0.0091 - val_mape: 19.6946 - val_mae: 0.0662
Epoch 49/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0020 - mape: 9907.6143 - mae: 0.0340 - val_loss: 0.0091 - val_mape: 20.3680 - val_mae: 0.0664
Epoch 50/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0019 - mape: 9755.1289 - mae: 0.0338 - val_loss: 0.0082 - val_mape: 18.6622 - val_mae: 0.0626
Epoch 51/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0019 - mape: 9754.2344 - mae: 0.0337 - val_loss: 0.0085 - val_mape: 19.3170 - val_mae: 0.0640
Epoch 52/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0019 - mape: 9523.7832 - mae: 0.0335 - val_loss: 0.0081 - val_mape: 19.9210 - val_mae: 0.0643
Epoch 53/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0019 - mape: 9164.6172 - mae: 0.0332 - val_loss: 

1105/1105 [==============================] - 8s 7ms/step - loss: 0.0011 - mape: 2363.7012 - mae: 0.0250 - val_loss: 0.0094 - val_mape: 21.7541 - val_mae: 0.0690
Epoch 96/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0011 - mape: 2810.0247 - mae: 0.0249 - val_loss: 0.0099 - val_mape: 22.4739 - val_mae: 0.0714
Epoch 97/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0011 - mape: 3665.3530 - mae: 0.0254 - val_loss: 0.0096 - val_mape: 21.9186 - val_mae: 0.0696
Epoch 98/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0010 - mape: 2397.6565 - mae: 0.0247 - val_loss: 0.0098 - val_mape: 22.0216 - val_mae: 0.0700
Epoch 99/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0010 - mape: 3617.1990 - mae: 0.0247 - val_loss: 0.0093 - val_mape: 21.5415 - val_mae: 0.0680
Epoch 100/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0010 - mape: 2807.1023 - mae: 0.0245 - val_loss: 0.0102 - val_

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 844.007395029068


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 6ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.23130906, 0.19442303, 0.18412563, ..., 0.47949338, 0.4316701 ,
       0.39298368], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.45526916, 0.41691211, 0.38233674, ..., 0.47147637, 0.43401972,
       0.40412643])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,338.3,213.933029
1,317.0,193.449905
2,297.8,187.731659
3,296.2,189.875168
4,292.7,198.539093
...,...,...
4363,385.7,388.185028
4364,366.2,373.764008
4365,347.3,351.751923
4366,326.5,325.195221


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

9.370008017473747